## --- Day 14: Chocolate Charts ---

The Elves are trying to come up with the ultimate hot chocolate recipe; they're even maintaining a scoreboard which tracks the quality score (0-9) of each recipe.

Only two recipes are on the board: the first recipe got a score of 3, the second, 7. Each of the two Elves has a current recipe: the first Elf starts with the first recipe, and the second Elf starts with the second recipe.

To create new recipes, the two Elves combine their current recipes. This creates new recipes from the digits of the sum of the current recipes' scores. With the current recipes' scores of 3 and 7, their sum is 10, and so two new recipes would be created: the first with score 1 and the second with score 0. If the current recipes' scores were 2 and 3, the sum, 5, would only create one recipe (with a score of 5) with its single digit.

The new recipes are added to the end of the scoreboard in the order they are created. So, after the first round, the scoreboard is 3, 7, 1, 0.

After all new recipes are added to the scoreboard, each Elf picks a new current recipe. To do this, the Elf steps forward through the scoreboard a number of recipes equal to 1 plus the score of their current recipe. So, after the first round, the first Elf moves forward 1 + 3 = 4 times, while the second Elf moves forward 1 + 7 = 8 times. If they run out of recipes, they loop back around to the beginning. After the first round, both Elves happen to loop around until they land on the same recipe that they had in the beginning; in general, they will move to different recipes.

Drawing the first Elf as parentheses and the second Elf as square brackets, they continue this process:

```
(3)[7]
(3)[7] 1  0 
 3  7  1 [0](1) 0 
 3  7  1  0 [1] 0 (1)
(3) 7  1  0  1  0 [1] 2 
 3  7  1  0 (1) 0  1  2 [4]
 3  7  1 [0] 1  0 (1) 2  4  5 
 3  7  1  0 [1] 0  1  2 (4) 5  1 
 3 (7) 1  0  1  0 [1] 2  4  5  1  5 
 3  7  1  0  1  0  1  2 [4](5) 1  5  8 
 3 (7) 1  0  1  0  1  2  4  5  1  5  8 [9]
 3  7  1  0  1  0  1 [2] 4 (5) 1  5  8  9  1  6 
 3  7  1  0  1  0  1  2  4  5 [1] 5  8  9  1 (6) 7 
 3  7  1  0 (1) 0  1  2  4  5  1  5 [8] 9  1  6  7  7 
 3  7 [1] 0  1  0 (1) 2  4  5  1  5  8  9  1  6  7  7  9 
 3  7  1  0 [1] 0  1  2 (4) 5  1  5  8  9  1  6  7  7  9  2 
```

The Elves think their skill will improve after making a few recipes (your puzzle input). However, that could take ages; you can speed this up considerably by identifying the scores of the ten recipes after that. For example:

If the Elves think their skill will improve after making 9 recipes, the scores of the ten recipes after the first nine on the scoreboard would be 5158916779 (highlighted in the last line of the diagram).

```
After 5 recipes, the scores of the next ten would be 0124515891.
After 18 recipes, the scores of the next ten would be 9251071085.
After 2018 recipes, the scores of the next ten would be 5941429882.
```

What are the scores of the ten recipes immediately after the number of recipes in your puzzle input?

Your puzzle input is `165061`.

### Part 1

In [1]:
import itertools
import numpy as np
from tqdm import tqdm
from collections import deque

PRECALC_DIVMOD = {x: divmod(x, 10) for x in range(10, 20)}


def run_fast(recipes0, positions0, preallocate=1_000_000):
    """
    highly tuned version that runs >65,000 iterations per second:
    - does not dynamically allocate any memory 
    - precalculates the possible new recipes when any modulo division is needed
    - more verbose to prevent memory copying
    """
    recipes = np.zeros(preallocate, dtype=np.uint8)
    r0, r1 = recipes0
    recipes[0], recipes[1] = recipes0
    
    i = 0
    idx_last = 1
    r0, r1 = recipes0
    p0, p1 = positions0
    while True:
        yield recipes[i]
        i += 1
        
        # see if we need to find new recipes
        if i > idx_last:
            r0 = recipes[p0]
            r1 = recipes[p1]
            
            # find new recipes
            s = r0 + r1
            if s < 10:
                recipes[idx_last+1] = s
                idx_last += 1
            else:
                recipes[idx_last+1:idx_last+3] = PRECALC_DIVMOD[s]
                idx_last += 2
            
            # move the elves
            p0 = (p0 + r0 + 1) % (idx_last + 1)
            p1 = (p1 + r1 + 1) % (idx_last + 1)
        
        # make sure we have enough memory allocated to keep going
        if (idx_last + 2) >= preallocate:
            recipes = np.append(recipes, np.zeros_like(recipes))
            

def n_after(recipes, positions, after, n=10):
    gen = run_fast(recipes, positions)
    for _ in tqdm(range(after)):
        next(gen)
    return ''.join([str(next(gen)) for _ in range(n)])

recipes = (3, 7)
positions = (0, 1)
assert n_after(recipes, positions, after=5, n=10) == '0124515891'
assert n_after(recipes, positions, after=18, n=10) == '9251071085'
assert n_after(recipes, positions, after=2018, n=10) == '5941429882'
n_after(recipes, positions, after=165061, n=10)

100%|██████████| 165061/165061 [00:01<00:00, 121451.32it/s]


'5992684592'

In [2]:
%load_ext line_profiler
%lprun -f run_fast n_after(recipes, positions, after=50000, n=10)

100%|██████████| 50000/50000 [00:01<00:00, 29042.45it/s]


Timer unit: 1e-06 s

Total time: 0.924714 s
File: <ipython-input-1-6793d6805da0>
Function: run_fast at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def run_fast(recipes0, positions0, preallocate=1_000_000):
    10                                               """
    11                                               highly tuned version that runs about 60,000 iterations per second:
    12                                               - does not dynamically allocate any memory 
    13                                               - 
    14                                               """
    15         1        348.0    348.0      0.0      recipes = np.zeros(preallocate, dtype=np.uint8)
    16         1          2.0      2.0      0.0      r0, r1 = recipes0
    17         1          7.0      7.0      0.0      recipes[0], recipes[1] = recipes0
    18                                               
    19         1  

### Part 2

As it turns out, you got the Elves' plan backwards. They actually want to know how many recipes appear on the scoreboard to the left of the first recipes whose scores are the digits from your puzzle input.

- 51589 first appears after 9 recipes.
- 01245 first appears after 5 recipes.
- 92510 first appears after 18 recipes.
- 59414 first appears after 2018 recipes.

How many recipes appear on the scoreboard to the left of the score sequence in your puzzle input?

In [3]:
def window(seq, n=2):
    """
    Returns a sliding window (of width n) over data from the iterable
       s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...
    from: https://docs.python.org/release/2.3.5/lib/itertools-example.html
    """
    it = iter(seq)
    result = tuple(itertools.islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def first_seen(recipes, positions, pattern, **kwargs):
    gen = run_fast(recipes, positions, **kwargs)
    n_at_a_time = window(gen, n=len(pattern))
    pbar = tqdm()
    for i, curr in enumerate(n_at_a_time):
        if curr == pattern:
            break
        pbar.update()
    pbar.close()
    return i

assert first_seen(recipes, positions, (5, 1, 5, 8, 9)) == 9
assert first_seen(recipes, positions, (0, 1, 2, 4, 5)) == 5
assert first_seen(recipes, positions, (9, 2, 5, 1, 0)) == 18
assert first_seen(recipes, positions, (5, 9, 4, 1, 4)) == 2018

9it [00:00, 9539.74it/s]
5it [00:00, 3974.14it/s]
18it [00:00, 28013.90it/s]
2018it [00:00, 60461.35it/s]


In [4]:
wanted = (1, 6, 5, 0, 6, 1)
first_seen(recipes, positions, wanted, preallocate=100_000_000)

20181148it [04:59, 67465.47it/s]


20181148